<a href="https://colab.research.google.com/github/abxda/COLMEX-ML/blob/main/Semana_12_RAG_COLMEX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!echo "--- Instalando paquetes necesarios ---"
!pip install -U duckduckgo_search wordcloud matplotlib pandas nltk spacy sentence-transformers flagembedding ollama chromadb seaborn scikit-learn --quiet
!echo "--- Descargando modelo spaCy en español ---"
!python -m spacy download es_core_news_sm --quiet
!echo "--- Instalaciones completadas ---"
!echo "NOTA: REINICIAR EL ENTORNO DE EJECUCIÓN."

In [ ]:
# ==============================================================================
# 2. IMPORTACIONES
# ==============================================================================
import os
import subprocess
import time
import string
import json

# Procesamiento de datos y NLP
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import spacy

# Búsqueda y Visualización
from duckduckgo_search import DDGS
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns

# Machine Learning: Vectorización, Clustering, Reducción de Dimensionalidad
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN, KMeans
from sklearn.preprocessing import normalize
from sklearn.manifold import TSNE

# Modelos de Embeddings
from FlagEmbedding import BGEM3FlagModel
from sentence_transformers import SentenceTransformer

# LLM y Base de Datos Vectorial
import ollama
import chromadb


In [ ]:
# ==============================================================================
# 3. CONFIGURACIÓN INICIAL Y DESCARGAS ADICIONALES
# ==============================================================================

# Descargar recursos de NLTK (stopwords y tokenizador punkt)
print("--- Descargando recursos de NLTK (stopwords, punkt) ---")
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
print("--- Recursos NLTK listos ---")
# Cargar modelo spaCy
print("--- Cargando modelo spaCy es_core_news_sm ---")
nlp = spacy.load("es_core_news_sm")
print("--- Modelo spaCy cargado ---")


# Modelos de Embedding (se cargarán más adelante justo antes de usarlos)
BGE_MODEL_NAME = 'BAAI/bge-m3'
E5_MODEL_NAME = 'intfloat/multilingual-e5-large'

# Modelos LLM (Ollama)
OLLAMA_MODEL_DEEPSEEK = 'deepseek-r1:8b' # Asegúrate de que este modelo esté disponible en tu instancia de Ollama
OLLAMA_MODEL_GEMMA = 'gemma3:12b'      # Asegúrate de que este modelo esté disponible
DEFAULT_OLLAMA_MODEL = OLLAMA_MODEL_GEMMA # Modelo a usar por defecto

# Nombre para la colección ChromaDB
CHROMA_COLLECTION_NAME = "news_rag_e5_clean"

# Político para análisis estructurado
POLITICO_ANALISIS = "Claudia Sheinbaum"

In [ ]:
# ==============================================================================
# 4. FUNCIONES AUXILIARES
# ==============================================================================

def buscar_noticias(keywords, region, max_results=100):
    """Busca noticias usando la versión síncrona de DDGS."""
    print(f"Buscando {max_results} noticias para '{keywords}' en región '{region}'...")
    with DDGS() as ddgs:
        noticias = list(ddgs.news(
            keywords=keywords,
            region=region,
            safesearch='moderate',
            max_results=max_results
        ))
    print(f"Se encontraron {len(noticias)} noticias.")
    return noticias

def limpiar_texto(text):
    """Convierte a minúsculas, quita puntuación y stopwords en español."""
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    palabras = text.split()
    stop_words_es = set(stopwords.words('spanish'))
    palabras_limpias = [word for word in palabras if word not in stop_words_es]
    return ' '.join(palabras_limpias)

def lematizar_texto(text, nlp_model):
    """Lematiza el texto usando un modelo spaCy."""
    if not isinstance(text, str):
        return ""
    doc = nlp_model(text)
    return ' '.join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

def generar_nube_de_palabras(textos, ax, titulo="Nube de Palabras"):
    """Genera y muestra una nube de palabras en un eje Matplotlib."""
    texto_completo = ' '.join(textos)
    if not texto_completo.strip():
        print(f"Advertencia: No hay texto para generar la nube de palabras '{titulo}'.")
        ax.text(0.5, 0.5, 'No hay datos suficientes', horizontalalignment='center', verticalalignment='center')
        ax.axis('off')
        return

    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(texto_completo)
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_title(titulo)
    ax.axis('off')

def visualizar_clusters(df, cluster_labels, textos_col, titulo_col, fuente_col, num_clusters_a_mostrar=None):
    """Visualiza clusters con nubes de palabras y títulos/fuentes."""
    df['cluster'] = cluster_labels
    cluster_counts = df['cluster'].value_counts()
    print("Conteo de noticias por cluster:")
    print(cluster_counts)

    # Filtrar clusters de ruido (-1 en DBSCAN) y limitar número si se especifica
    clusters_validos = cluster_counts[cluster_counts.index != -1].index
    if num_clusters_a_mostrar is not None:
        clusters_validos = clusters_validos[:num_clusters_a_mostrar]

    num_clusters = len(clusters_validos)
    if num_clusters == 0:
        print("No se encontraron clusters válidos para visualizar.")
        return

    fig, axs = plt.subplots(num_clusters, 2, figsize=(20, 6 * num_clusters))
    # Asegurarse que axs es siempre un array 2D
    if num_clusters == 1:
        axs = np.array([axs])

    for i, cluster_id in enumerate(clusters_validos):
        cluster_data = df[df['cluster'] == cluster_id]
        textos_cluster = cluster_data[textos_col].tolist()
        titulos_fuentes_cluster = cluster_data[[titulo_col, fuente_col]].apply(
            lambda x: f"- {x[titulo_col]} ({x[fuente_col]})", axis=1
        ).tolist()

        # Generar nube de palabras
        generar_nube_de_palabras(textos_cluster, axs[i, 0], titulo=f'Cluster {cluster_id} - Nube de Palabras')

        # Mostrar títulos y fuentes
        axs[i, 1].text(0.05, 0.95, "\n".join(titulos_fuentes_cluster),
                       verticalalignment='top', horizontalalignment='left',
                       fontsize=9, family='monospace', wrap=True)
        axs[i, 1].set_title(f'Cluster {cluster_id} - Títulos y Fuentes ({len(titulos_fuentes_cluster)} noticias)')
        axs[i, 1].axis('off')

    plt.tight_layout(pad=3.0)
    plt.show()

def visualizar_tsne(X_embedded, df_metadata, titulo_col, plot_title):
    """Realiza t-SNE y visualiza los embeddings con etiquetas."""
    print(f"--- Realizando t-SNE para {plot_title} ---")
    if X_embedded.shape[0] <= 1:
        print("Se necesita más de 1 punto de datos para t-SNE.")
        return
    # Ajustar perplexity si hay pocos datos
    perplexity_value = min(30, X_embedded.shape[0] - 1)

    tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity_value, n_iter=300) # Reducir iteraciones si es lento
    X_2d = tsne.fit_transform(X_embedded)

    df_tsne = pd.DataFrame(X_2d, columns=['x', 'y'], index=df_metadata.index)
    df_tsne['label'] = df_metadata[titulo_col] # Asegúrate que titulo_col es el nombre correcto ('title')

    plt.figure(figsize=(18, 12))
    scatter = sns.scatterplot(data=df_tsne, x='x', y='y', hue='label', palette='viridis', legend=False, alpha=0.7, s=50)

    # --- INICIO DE LA SECCIÓN CORREGIDA ---
    # Añadir etiquetas (puede ser lento/denso con muchas noticias)
    # Considera mostrar solo una fracción o en hover interactivo en otros entornos
    for i in df_tsne.index:
        plt.text(df_tsne.loc[i, 'x'] + 0.05, df_tsne.loc[i, 'y'] + 0.05, # Pequeño offset
                 df_tsne.loc[i, 'label'][:50] + "...", # Acortar etiqueta a 50 caracteres
                 fontsize=8, alpha=0.7)
    # --- FIN DE LA SECCIÓN CORREGIDA ---

    plt.title(plot_title)
    plt.xlabel('Componente t-SNE 1')
    plt.ylabel('Componente t-SNE 2')
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.show()


In [ ]:
# Configuraciones para la búsqueda de noticias
configuraciones = [
    {"idioma": "Español", "keywords": "México", "region": "mx-es"}
]

In [ ]:
# ==============================================================================
# 5. BÚSQUEDA Y PROCESAMIENTO INICIAL DE NOTICIAS
# ==============================================================================

# Realizar la búsqueda
todas_las_noticias = []
for conf in configuraciones:
    noticias = buscar_noticias(conf['keywords'], conf['region'])
    todas_las_noticias.extend(noticias)
    print(f"Resultados acumulados: {len(todas_las_noticias)} noticias.")

# Crear DataFrame
df_noticias = pd.DataFrame(todas_las_noticias)

if not df_noticias.empty and 'body' in df_noticias.columns and 'title' in df_noticias.columns:
    print("\n--- DataFrame inicial de noticias ---")
    print(df_noticias.head())
    print(f"\nDimensiones del DataFrame: {df_noticias.shape}")

    # Limpieza básica y lematización
    print("\n--- Limpiando y Lematizando texto ---")
    df_noticias['clean_body'] = df_noticias['body'].apply(limpiar_texto)
    df_noticias['lemmatized_body'] = df_noticias['clean_body'].apply(lambda x: lematizar_texto(x, nlp))

    print("\n--- DataFrame con texto limpio y lematizado ---")
    print(df_noticias[['title', 'clean_body', 'lemmatized_body']].head())

    # Generar Nube de Palabras Inicial (sobre texto lematizado)
    print("\n--- Generando Nube de Palabras (Lematizada) ---")
    fig_wc, ax_wc = plt.subplots(1, 1, figsize=(10, 5))
    generar_nube_de_palabras(df_noticias['lemmatized_body'].dropna(), ax_wc, "Nube de Palabras Global (Lematizada)")
    plt.show()

    # Análisis de Fuentes
    print("\n--- Analizando distribución de fuentes ---")
    if 'source' in df_noticias.columns:
        conteo_fuentes = df_noticias['source'].value_counts()
        plt.figure(figsize=(12, 6))
        top_n = 20 # Mostrar solo las top N fuentes
        conteo_fuentes.head(top_n).plot(kind='bar', color='skyblue')
        plt.title(f'Top {top_n} Fuentes de Noticias')
        plt.xlabel('Fuente')
        plt.ylabel('Cantidad de Noticias')
        plt.xticks(rotation=75, ha='right')
        plt.tight_layout()
        plt.show()
    else:
        print("Advertencia: Columna 'source' no encontrada para análisis de fuentes.")

else:
    print("\n--- No se encontraron noticias o el DataFrame no tiene las columnas 'body'/'title'. Saltando análisis detallado. ---")
    # Salir o manejar el caso de no noticias si es necesario
    exit() # O gestionarlo de otra forma

In [ ]:
# ==============================================================================
# 6. CLUSTERING CON TF-IDF
# ==============================================================================
print("\n" + "="*60)
print(" 6. CLUSTERING CON TF-IDF (N-gramas de caracteres)")
print("="*60)

vectorizer_tfidf = TfidfVectorizer(analyzer='char', ngram_range=(3, 5))
X_tfidf = vectorizer_tfidf.fit_transform(df_noticias['lemmatized_body'].dropna())
X_tfidf_normalized = normalize(X_tfidf)

print(f"Dimensiones de la matriz TF-IDF: {X_tfidf_normalized.shape}")

# --- Clustering con K-Means (TF-IDF) ---
print("\n--- Clustering con K-Means (TF-IDF) ---")
kmeans_tfidf = KMeans(n_clusters=8, random_state=42, n_init=10)
clusters_kmeans_tfidf = kmeans_tfidf.fit_predict(X_tfidf_normalized)
visualizar_clusters(df_noticias, clusters_kmeans_tfidf, 'lemmatized_body', 'title', 'source', num_clusters_a_mostrar=8)

# --- Clustering con DBSCAN (TF-IDF) ---
print("\n--- Clustering con DBSCAN (TF-IDF) ---")
# Nota: eps puede necesitar ajuste según los datos
dbscan_tfidf = DBSCAN(eps=0.8, min_samples=2, metric='cosine')
clusters_dbscan_tfidf = dbscan_tfidf.fit_predict(X_tfidf_normalized)
visualizar_clusters(df_noticias, clusters_dbscan_tfidf, 'lemmatized_body', 'title', 'source')

In [ ]:
# ==============================================================================
# 7. CLUSTERING CON EMBEDDINGS BGE-M3
# ==============================================================================
print("\n" + "="*60)
print(f" 7. CLUSTERING CON EMBEDDINGS BGE-M3 ({BGE_MODEL_NAME})")
print("="*60)

print(f"--- Cargando modelo BGE: {BGE_MODEL_NAME} ---")
model_bge = BGEM3FlagModel(BGE_MODEL_NAME) # Carga puede tardar
print("--- Modelo BGE cargado ---")

print("--- Generando embeddings BGE-M3 para 'clean_body' ---")
embeddings_bge_list = []
texts_to_embed_bge = df_noticias['clean_body'].dropna().tolist()
if texts_to_embed_bge:
     # BGE espera una lista, encode devuelve dict con 'dense_vecs'
    embeddings_bge = model_bge.encode(texts_to_embed_bge, batch_size=12, max_length=512)['dense_vecs'] # Ajustar batch_size según memoria
    X_bge = np.array(embeddings_bge)
    print(f"Dimensiones de los embeddings BGE: {X_bge.shape}")

    # --- Clustering con K-Means (BGE) ---
    print("\n--- Clustering con K-Means (BGE) ---")
    kmeans_bge = KMeans(n_clusters=8, random_state=42, n_init=10)
    clusters_kmeans_bge = kmeans_bge.fit_predict(X_bge) # BGE ya suele estar normalizado
    visualizar_clusters(df_noticias.dropna(subset=['clean_body']), clusters_kmeans_bge, 'clean_body', 'title', 'source', num_clusters_a_mostrar=8)

    # --- Clustering con DBSCAN (BGE) ---
    print("\n--- Clustering con DBSCAN (BGE) ---")

    dbscan_bge = DBSCAN(eps=0.365, min_samples=2, metric='cosine') # Usar cosine suele ser mejor para high-dim embeddings
    clusters_dbscan_bge = dbscan_bge.fit_predict(X_bge)
    visualizar_clusters(df_noticias.dropna(subset=['clean_body']), clusters_dbscan_bge, 'clean_body', 'title', 'source')

    # --- Visualización t-SNE (BGE) ---
    visualizar_tsne(X_bge, df_noticias.dropna(subset=['clean_body']), 'title', 't-SNE de Embeddings BGE-M3')

else:
    print("No hay textos limpios para generar embeddings BGE.")


In [ ]:
# ==============================================================================
# 8. CLUSTERING CON EMBEDDINGS MULTILINGUAL-E5
# ==============================================================================
print("\n" + "="*60)
print(f" 8. CLUSTERING CON EMBEDDINGS E5 ({E5_MODEL_NAME})")
print("="*60)

print(f"--- Cargando modelo E5: {E5_MODEL_NAME} ---")
model_e5 = SentenceTransformer(E5_MODEL_NAME)
print("--- Modelo E5 cargado ---")

print("--- Generando embeddings E5 para 'clean_body' (con prefijo 'passage:') ---")
texts_to_embed_e5_raw = df_noticias['clean_body'].dropna().tolist()
if texts_to_embed_e5_raw:
    texts_to_embed_e5 = ["passage: " + text for text in texts_to_embed_e5_raw]
    embeddings_e5 = model_e5.encode(texts_to_embed_e5, show_progress_bar=True, batch_size=32) # Ajustar batch_size
    X_e5 = np.array(embeddings_e5)
    X_e5_normalized = normalize(X_e5, norm='l2', axis=1) # E5 recomienda normalizar
    print(f"Dimensiones de los embeddings E5 normalizados: {X_e5_normalized.shape}")

    # --- Clustering con K-Means (E5) ---
    print("\n--- Clustering con K-Means (E5) ---")
    kmeans_e5 = KMeans(n_clusters=8, random_state=42, n_init=10)
    clusters_kmeans_e5 = kmeans_e5.fit_predict(X_e5_normalized)
    visualizar_clusters(df_noticias.dropna(subset=['clean_body']), clusters_kmeans_e5, 'clean_body', 'title', 'source', num_clusters_a_mostrar=8)

    # --- Clustering con DBSCAN (E5) ---
    print("\n--- Clustering con DBSCAN (E5) ---")
    # Eps ajustado basado en el ejemplo original, puede necesitar tuning
    dbscan_e5 = DBSCAN(eps=0.365, min_samples=2, metric='cosine') # Usar cosine con normalizados
    clusters_dbscan_e5 = dbscan_e5.fit_predict(X_e5_normalized)
    visualizar_clusters(df_noticias.dropna(subset=['clean_body']), clusters_dbscan_e5, 'clean_body', 'title', 'source')

    # --- Visualización t-SNE (E5) ---
    visualizar_tsne(X_e5_normalized, df_noticias.dropna(subset=['clean_body']), 'title', f't-SNE de Embeddings {E5_MODEL_NAME}')

else:
    print("No hay textos limpios para generar embeddings E5.")


In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!pip install -q ollama chromadb

In [ ]:
print("--- Configurando y iniciando Ollama Server ---")
# Lanzar el servidor Ollama en segundo plano
# Usamos nohup y & para que siga corriendo aunque cerremos la terminal (o se desconecte Colab)
ollama_process = subprocess.Popen(['nohup', 'ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Darle tiempo al servidor para que inicie
print("Esperando a que el servidor Ollama inicie (10 segundos)...")
time.sleep(10)


In [ ]:
OLLAMA_MODEL_GEMMA

In [ ]:

# ==============================================================================
# 9. CONFIGURACIÓN DE OLLAMA Y CHROMADB PARA RAG
# ==============================================================================
print("\n" + "="*60)
print(" 9. CONFIGURACIÓN DE OLLAMA Y CHROMADB PARA RAG")
print("="*60)

print("--- Verificando conexión con Ollama ---")
try:
    ollama.list()
    print("Ollama server detectado.")
except Exception as e:
    print(f"ERROR: No se pudo conectar con Ollama. Asegúrate de que 'ollama serve' esté corriendo. Detalle: {e}")
    # Podrías decidir salir aquí si Ollama es esencial
    # exit()

# --- Descargar Modelos Ollama (si no existen) ---
def pull_ollama_model(model_name):
    print(f"Verificando/Descargando modelo Ollama: {model_name}...")
    try:
        # Verificar si el modelo existe localmente
        model_list = ollama.list()['models']
        if any(m['model'] == model_name for m in model_list):
            print(f"Modelo '{model_name}' ya existe localmente.")
            return True

        # Si no existe, intentar descargarlo
        print(f"Descargando modelo '{model_name}' (puede tardar)...")
        # El pull a través de la librería a veces es menos informativo que el CLI
        current_digest = ""
        for progress in ollama.pull(model_name, stream=True):
            digest = progress.get("digest", "")
            if digest != current_digest and digest != "":
                print(f"Pulling {model_name}: {digest} - {progress.get('completed', 0)}/{progress.get('total', 0)}")
                current_digest = digest
            elif 'status' in progress and progress['status'] == 'success':
                 print(f"Descarga de '{model_name}' completada.")
                 return True
            elif 'error' in progress:
                 print(f"ERROR descargando '{model_name}': {progress['error']}")
                 return False
        # Si el stream termina sin 'success' o 'error' explícito (raro)
        print(f"Descarga de '{model_name}' finalizada (estado final desconocido desde stream). Verifique manualmente.")
        return True # Asumir éxito si no hubo error explícito

    except Exception as e:
        print(f"ERROR durante la descarga/verificación del modelo '{model_name}': {e}")
        return False




In [ ]:
# Descargar los modelos necesarios
model_download_ok = pull_ollama_model(OLLAMA_MODEL_DEEPSEEK)

In [ ]:
model_download_ok = pull_ollama_model(OLLAMA_MODEL_GEMMA) and model_download_ok

In [ ]:
ollama.list()['models']

In [ ]:
# --- Configurar ChromaDB e Indexar Datos ---
print("\n--- Configurando ChromaDB e Indexando Noticias ---")
data_available_for_rag = False
collection = None
embedding_model_for_rag = model_e5 # Usaremos E5 para RAG como en el original

texts_for_rag = df_noticias['clean_body'].dropna().tolist()
ids_for_rag = [f"news_{i}" for i, _ in enumerate(texts_for_rag)]

if texts_for_rag:
    print(f"Se prepararon {len(texts_for_rag)} textos de noticias para indexar.")

    print("Generando embeddings para RAG (usando E5 con prefijo 'passage:')...")
    texts_with_prefix_rag = ["passage: " + text for text in texts_for_rag]
    try:
        news_embeddings_rag = embedding_model_for_rag.encode(
            texts_with_prefix_rag,
            show_progress_bar=True,
            batch_size=32
        )

        print("Inicializando ChromaDB (cliente en memoria)...")
        chroma_client = chromadb.Client() # O PersistentClient(path="ruta/a/db") para persistencia

        # Crear o cargar colección (borrándola si existe para limpieza en este script)
        print(f"Intentando crear/recrear colección ChromaDB: '{CHROMA_COLLECTION_NAME}'")
        try:
            chroma_client.delete_collection(name=CHROMA_COLLECTION_NAME)
            print(f"Colección existente '{CHROMA_COLLECTION_NAME}' eliminada.")
        except Exception:
            print(f"Colección '{CHROMA_COLLECTION_NAME}' no existía previamente o no se pudo borrar.")

        collection = chroma_client.create_collection(
            name=CHROMA_COLLECTION_NAME,
            metadata={"hnsw:space": "cosine"}
            )

        print("Añadiendo documentos a ChromaDB...")
        collection.add(
            embeddings=news_embeddings_rag.tolist(), # Chroma espera listas
            documents=texts_for_rag, # Guardamos el texto limpio SIN prefijo
            ids=ids_for_rag
        )
        print(f"--- Datos indexados ({collection.count()} documentos) en ChromaDB. Colección '{CHROMA_COLLECTION_NAME}' lista. ---")
        data_available_for_rag = True
    except Exception as e:
        print(f"ERROR durante la generación de embeddings o indexación en ChromaDB: {e}")
        data_available_for_rag = False

else:
    print("--- ADVERTENCIA: No hay textos limpios ('clean_body') disponibles en df_noticias. La función RAG no tendrá contexto. ---")
    data_available_for_rag = False

In [ ]:
# ==============================================================================
# 10. FUNCIÓN RAG Y EJEMPLOS
# ==============================================================================
print("\n" + "="*60)
print(" 10. FUNCIÓN RAG (Retrieval-Augmented Generation)")
print("="*60)

def ask_llm_with_rag(query: str, ollama_model_name: str, embed_model, db_collection, k_results: int = 3):
    """Realiza una consulta RAG: recupera contexto de ChromaDB y pregunta a Ollama."""
    print(f"\n--- Procesando consulta RAG para: '{query}' ---")
    print(f"--- Usando LLM: {ollama_model_name} ---")

    if db_collection is None or not data_available_for_rag:
        print("--- ADVERTENCIA: Base de datos no disponible o vacía. Preguntando directamente al modelo sin contexto. ---")
        context_text = "No hay contexto disponible."
    else:
        # 1. Embeddear la consulta (con prefijo "query:")
        print("Generando embedding para la consulta...")
        try:
            query_embedding = embed_model.encode(["query: " + query])[0]

            # 2. Buscar en ChromaDB
            print(f"Buscando {k_results} documentos relevantes en ChromaDB...")
            results = db_collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=k_results,
                include=['documents', 'distances'] # Pedir distancias puede ser útil para debug
            )

            # 3. Extraer contexto y mostrar info
            retrieved_docs = results['documents'][0]
            distances = results['distances'][0]
            print("--- Contexto Recuperado ---")
            context_text = ""
            for i, doc in enumerate(retrieved_docs):
                dist_str = f"{distances[i]:.4f}"
                print(f"  Doc {i+1} (Distancia: {dist_str}): {doc[:150]}...") # Mostrar inicio del doc
                context_text += f"Fragmento {i+1}:\n{doc}\n\n"
            context_text = context_text.strip()
            if not context_text:
                print("Advertencia: No se recuperó ningún documento relevante.")
                context_text = "No se encontró contexto relevante en la base de datos."

        except Exception as e:
            print(f"ERROR durante la recuperación de contexto: {e}")
            context_text = "Error al recuperar contexto."

    # 4. Construir el prompt para Ollama
    prompt = f"""Eres un asistente que responde preguntas basándose ESTRICTAMENTE en el contexto proporcionado.
Si la información necesaria para responder la pregunta no se encuentra en el contexto, debes indicar CLARAMENTE que no puedes responder con la información dada. No inventes información ni utilices conocimiento externo.

Contexto Recuperado de Noticias:
--- START OF CONTEXT ---
{context_text}
--- END OF CONTEXT ---

Pregunta del Usuario: {query}

Respuesta Basada Únicamente en el Contexto:"""

    print(f"--- Enviando pregunta y contexto a {ollama_model_name} vía Ollama... ---")

    # 5. Llamar a Ollama
    try:
        response = ollama.chat(
            model=ollama_model_name,
            messages=[{'role': 'user', 'content': prompt}],
            options={'temperature': 0.1} # Baja temperatura para respuestas basadas en hechos
        )
        final_answer = response['message']['content']
    except Exception as e:
        print(f"ERROR al llamar a Ollama ({ollama_model_name}): {e}")
        final_answer = f"Error al obtener respuesta del modelo {ollama_model_name}."

    # 6. Devolver la respuesta del modelo
    return final_answer

# --- Ejemplos de Uso RAG ---
if data_available_for_rag:
    user_question = "¿Qué acciones o declaraciones se mencionan sobre Claudia Sheinbaum en relación con los aranceles o tensiones comerciales entre México y EE.UU.?"

    # Ejemplo con Gemma 3
    print("\n--- Ejecutando RAG con Gemma 3 ---")
    final_answer_gemma = ask_llm_with_rag(
        query=user_question,
        ollama_model_name=OLLAMA_MODEL_GEMMA,
        embed_model=embedding_model_for_rag,
        db_collection=collection,
        k_results=5
    )
    print("\n--- Respuesta Final de Gemma 3 (RAG) ---")
    print(final_answer_gemma)

    # Ejemplo con DeepSeek (si se descargó)
    if any(m['model'] == OLLAMA_MODEL_DEEPSEEK for m in ollama.list()['models']):
        print("\n--- Ejecutando RAG con DeepSeek ---")
        final_answer_deepseek = ask_llm_with_rag(
            query=user_question,
            ollama_model_name=OLLAMA_MODEL_DEEPSEEK,
            embed_model=embedding_model_for_rag,
            db_collection=collection,
            k_results=5
        )
        print("\n--- Respuesta Final de DeepSeek (RAG) ---")
        print(final_answer_deepseek)
    else:
        print(f"\n--- Modelo {OLLAMA_MODEL_DEEPSEEK} no encontrado, saltando ejemplo RAG con DeepSeek. ---")

else:
    # Si no hay datos, hacer una pregunta directa como demostración
    print("\n--- No hay datos indexados para RAG. Haciendo una pregunta directa al LLM ---")
    user_question_direct = "¿Qué son los aranceles comerciales?"
    try:
        response_direct = ollama.chat(
            model=DEFAULT_OLLAMA_MODEL, # Usar el modelo por defecto
            messages=[{'role': 'user', 'content': user_question_direct}]
        )
        print(f"\n--- Respuesta Final de {DEFAULT_OLLAMA_MODEL} (Directa) ---")
        print(response_direct['message']['content'])
    except Exception as e:
        print(f"ERROR al hacer pregunta directa a {DEFAULT_OLLAMA_MODEL}: {e}")

In [ ]:
# ==============================================================================
# 11. EXTRACCIÓN ESTRUCTURADA DE INFORMACIÓN
# ==============================================================================
print("\n" + "="*60)
print(f" 11. EXTRACCIÓN ESTRUCTURADA DE INFORMACIÓN (Enfoque: {POLITICO_ANALISIS})")
print("="*60)
DEFAULT_OLLAMA_MODEL = OLLAMA_MODEL_DEEPSEEK
structured_results = []

# Verificar si df_noticias existe y tiene las columnas necesarias
if 'df_noticias' in locals() and not df_noticias.empty and 'title' in df_noticias.columns and 'body' in df_noticias.columns:
    print(f"Procesando noticias para análisis estructurado sobre: {POLITICO_ANALISIS}")
    print(f"Usando LLM: {DEFAULT_OLLAMA_MODEL}")

    # Limitar a N noticias para el ejemplo, quitar .head() para procesar todo
    # ADVERTENCIA: Procesar todo puede tardar MUCHO tiempo y consumir recursos.
    num_noticias_a_procesar = 5
    print(f"Se procesarán las primeras {num_noticias_a_procesar} noticias.")

    for index, row in df_noticias.head(num_noticias_a_procesar).iterrows():
        news_title = row['title']
        news_text = row['body'] # Usar el cuerpo original para más contexto
        news_id = f"news_{index}"

        print(f"\n--- Procesando Noticia #{index}: {news_title[:60]}... ---")

        if not news_text or not isinstance(news_text, str):
             print("  Advertencia: Cuerpo de la noticia vacío o inválido. Saltando.")
             structured_results.append({
                 'indice_noticia': index, 'titulo': news_title,
                 f'favor_{POLITICO_ANALISIS.split()[0]}': ["Texto de noticia inválido"],
                 f'contra_{POLITICO_ANALISIS.split()[0]}': ["Texto de noticia inválido"],
                 'aspectos_clave': ["Texto de noticia inválido"],
                 'respuesta_llm_cruda': "N/A - Texto inválido"
             })
             continue

        # Crear el prompt para la extracción estructurada en JSON
        prompt_template = f"""
        Analiza el siguiente texto de noticia con un enfoque específico en la figura política: '{POLITICO_ANALISIS}'.
        Extrae la siguiente información y preséntala ESTRICTAMENTE en formato JSON. Asegúrate de que el JSON esté bien formado y sea el ÚNICO contenido de tu respuesta.

        Formato JSON esperado:
        {{
          "puntos_a_favor": ["lista de frases textuales del artículo que sean favorables a {POLITICO_ANALISIS}"],
          "puntos_en_contra": ["lista de frases textuales del artículo que sean críticas o desfavorables a {POLITICO_ANALISIS}"],
          "menciones_neutrales": ["lista de frases textuales donde se menciona a {POLITICO_ANALISIS} sin una clara connotación positiva o negativa"],
          "resumen_general": "Un resumen muy breve (1-2 frases) del contenido principal de la noticia, independientemente de si menciona a {POLITICO_ANALISIS}."
        }}

        Instrucciones importantes:
        - Basa tu análisis ÚNICAMENTE en el texto proporcionado. No añadas información externa ni opiniones.
        - Si '{POLITICO_ANALISIS}' no es mencionado en absoluto, las listas 'puntos_a_favor', 'puntos_en_contra' y 'menciones_neutrales' deben ser listas vacías []. El resumen general aún debe proporcionarse.
        - Si hay menciones pero no son claramente favorables o desfavorables, ponlas en 'menciones_neutrales'.
        - Si no hay puntos claros a favor o en contra, devuelve listas vacías [] para esas claves.
        - Tu respuesta DEBE ser SÓLO el objeto JSON, sin ningún texto introductorio o explicativo antes o después.

        Texto de la Noticia:
        --- START OF TEXT ---
        {news_text}
        --- END OF TEXT ---

        JSON Output:
        """

        # Llamar a Ollama
        structured_data = {}
        llm_raw_output = ""
        try:
            response = ollama.chat(
                model=DEFAULT_OLLAMA_MODEL,
                messages=[{'role': 'user', 'content': prompt_template}],
                options={'temperature': 0.0}, # Temperatura cero para máxima consistencia
                format="json" # ¡Pedir formato JSON directamente a Ollama si la versión lo soporta!
            )
            llm_raw_output = response['message']['content']
            print(f"  Respuesta cruda del LLM recibida.")# Primeros 100 chars: {llm_raw_output[:100]}...")

            # Intentar parsear la respuesta JSON del LLM
            # ADVERTENCIA: Sin try-except robusto, esto fallará si la respuesta no es JSON válido.
            # El format="json" ayuda mucho pero no es 100% garantizado.
            # Minimal check for JSON structure
            json_string = llm_raw_output.strip()
            if json_string.startswith('{') and json_string.endswith('}'):
                 structured_data = json.loads(json_string)
                 print("  JSON parseado con éxito.")
            else:
                 print("  ADVERTENCIA: La respuesta del LLM no parece ser un JSON válido completo.")
                 print(f"  Respuesta recibida: {llm_raw_output}")
                 # Intentar encontrar JSON dentro (menos fiable)
                 json_start = llm_raw_output.find('{')
                 json_end = llm_raw_output.rfind('}') + 1
                 if json_start != -1 and json_end != -1:
                     try:
                         json_string_extracted = llm_raw_output[json_start:json_end]
                         structured_data = json.loads(json_string_extracted)
                         print("  JSON extraído y parseado con éxito desde la respuesta.")
                     except json.JSONDecodeError as e_extract:
                         print(f"  ERROR: No se pudo decodificar el JSON extraído. Error: {e_extract}")
                         structured_data = {"error": "JSON Decode Error after extraction", "raw_output": llm_raw_output}
                 else:
                    structured_data = {"error": "Valid JSON object not found in response", "raw_output": llm_raw_output}


        except Exception as e:
            print(f"  ERROR durante la llamada a Ollama o el parseo JSON: {e}")
            structured_data = {"error": str(e), "raw_output": llm_raw_output} # Guardar error

        # Extraer la información usando .get() para manejar claves faltantes o errores
        favor = structured_data.get('puntos_a_favor', ["Error/No encontrado"])
        contra = structured_data.get('puntos_en_contra', ["Error/No encontrado"])
        neutral = structured_data.get('menciones_neutrales', ["Error/No encontrado"])
        resumen = structured_data.get('resumen_general', "Error/No encontrado")

        # Guardar los resultados
        structured_results.append({
            'indice_noticia': index,
            'titulo': news_title,
            f'favor_{POLITICO_ANALISIS.split()[0]}': favor,
            f'contra_{POLITICO_ANALISIS.split()[0]}': contra,
            f'neutral_{POLITICO_ANALISIS.split()[0]}': neutral,
            'resumen_noticia': resumen,
            'respuesta_llm_cruda': llm_raw_output # Guardar para referencia/debug
        })

    # Crear DataFrame final con la información estructurada
    print("\n--- Creando DataFrame con información estructurada ---")
    df_structured_analysis = pd.DataFrame(structured_results)

    # Mostrar el DataFrame resultante
    pd.set_option('display.max_colwidth', 100) # Ajustar ancho de columna
    pd.set_option('display.max_rows', 50) # Ajustar filas mostradas
    print(df_structured_analysis)

else:
    print("--- No se puede realizar el análisis estructurado: 'df_noticias' no está disponible o no tiene las columnas 'title' y 'body'. ---")

print("\n" + "="*60)
print("--- PROCESO COMPLETADO ---")
print("="*60)

In [ ]:
df_structured_analysis